In [38]:
from argparse import ArgumentParser
import sys, os
import torch

sys.path.append('../')

In [42]:
model_param, entropy_param, iter_num = torch.load(
    "../output/trial/CompressedTrainer_BaseGaussian21000_lmbda_1e-08.pth",
    map_location="cpu"
)

In [43]:
(
    active_sh_degree,
    _xyz,
    _features_dc,
    _features_rest,
    _scaling,
    _rotation,
    _opacity,
    max_radii2D,
    xyz_gradient_accum,
    denom,
    spatial_lr_scale,
) = model_param[:-1]

In [44]:
print(_features_dc.shape)
print(_features_rest.shape)

torch.Size([810226, 1, 3])
torch.Size([810226, 15, 3])


In [50]:
def concat_spherical_harmonics(_features_dc, _features_rest):
    spherical_harmonics_reshaped = torch.permute(
        torch.cat(
            (_features_dc, _features_rest), dim=1
        ).flatten(1, 2),
        dims=(1, 0),
    ).unsqueeze(0)

    return spherical_harmonics_reshaped

In [52]:
em = EntropyBottleneck(48)
em.update()
em.restore_model(entropy_param[0])

x = concat_spherical_harmonics(_features_dc, _features_rest)

Successfully loaded EntropyBottleneck


In [71]:
em._quantized_cdf[4]

tensor([    0,     1,     2,     3,     4,     5,     6,    15,  1069, 64496,
        65528, 65529, 65530, 65531, 65532, 65533, 65534, 65535, 65536,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0], dtype=torch.int32)

In [68]:
em._cdf_length

tensor([77, 77, 73, 21, 19, 21, 21, 21, 21, 21, 21, 21, 19, 20, 19, 21, 21, 21,
        21, 21, 21, 21, 20, 20, 20, 21, 19, 21, 19, 21, 19, 21, 21, 21, 20, 19,
        21, 21, 21, 20, 21, 21, 21, 21, 21, 21, 21, 21], dtype=torch.int32)

In [86]:
indexes = em._build_indexes(x.size())
medians = em._get_medians().detach()
spatial_dims = len(x.size()) - 2
medians = em._extend_ndims(medians, spatial_dims)
medians = medians.expand(x.size(0), *([-1] * (spatial_dims + 1)))
# return super().compress(x, indexes, medians)

In [87]:
medians.shape

torch.Size([1, 48, 1])

In [88]:
symbols = em.quantize(x, "symbols", medians)
symbols

tensor([[[-1,  0,  1,  ...,  1,  2, -1],
         [ 0,  0,  1,  ...,  1,  1, -1],
         [ 0,  0,  2,  ...,  1,  1, -1],
         ...,
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0]]], dtype=torch.int32)

In [75]:
indexes[0].reshape(-1)

tensor([ 0,  0,  0,  ..., 47, 47, 47], dtype=torch.int32)

In [63]:
len(symbols.size())

3

In [64]:
em._check_cdf_size()